In [1]:
import rmcalyse
from rmcalyse import orth_labels, orthonormal_positions
from plugins.distance_calculations import distance_calculator



 8888888b. 888b     d888 .d8888b.         888                        
 888   Y88b8888b   d8888d88P  Y88b        888                        
 888    88888888b.d88888888    888        888                        
 888   d88P888Y88888P888888        8888b. 888888  888.d8888b  .d88b. 
 8888888P" 888 Y888P 888888           "88b888888  88888K     d8P  Y8b
 888 T88b  888  Y8P  888888    888.d888888888888  888"Y8888b.88888888
 888  T88b 888   "   888Y88b  d88P888  888888Y88b 888     X88Y8b.    
 888   T88b888       888 "Y8888P" "Y888888888 "Y88888 88888P' "Y8888 
                                                   888               
                                              Y8b d88P               
                                               "Y88P"                
 ------------- A toolbox for analysing RMCProfile outputs -----------
 --------------------------------------------------------------------
 ------------------------ Anton Goetzee-Barral ----------------------
 -----------------

In [2]:
# test distance calc import is working 
import numpy as np
a = np.array([0,0,0])
b = np.array([3,4,12])
distance_calculator.eucledian_distance(a,b) == 13

True

In [3]:
orth_labels, orthonormal_positions

(array([['1', 'Sr'],
        ['2', 'Sr'],
        ['3', 'Sr'],
        ['4', 'Sr'],
        ['5', 'Sr'],
        ['6', 'Sr'],
        ['7', 'Sr'],
        ['8', 'Sr'],
        ['9', 'Sr'],
        ['10', 'Sr'],
        ['11', 'Sr'],
        ['12', 'Sr'],
        ['13', 'Sr'],
        ['14', 'Sr'],
        ['15', 'Sr'],
        ['16', 'Sr'],
        ['17', 'Sr'],
        ['18', 'Sr'],
        ['19', 'Sr'],
        ['20', 'Sr'],
        ['21', 'Sr'],
        ['22', 'Sr'],
        ['23', 'Sr'],
        ['24', 'Sr'],
        ['25', 'Sr'],
        ['26', 'Sr'],
        ['27', 'Sr'],
        ['28', 'Ti'],
        ['29', 'Ti'],
        ['30', 'Ti'],
        ['31', 'Ti'],
        ['32', 'Ti'],
        ['33', 'Ti'],
        ['34', 'Ti'],
        ['35', 'Ti'],
        ['36', 'Ti'],
        ['37', 'Ti'],
        ['38', 'Ti'],
        ['39', 'Ti'],
        ['40', 'Ti'],
        ['41', 'Ti'],
        ['42', 'Ti'],
        ['43', 'Ti'],
        ['44', 'Ti'],
        ['45', 'Ti'],
        ['46', 'Ti'

In [ ]:
center_atom = ['Ti']
orbit_atom = ['O']
max_d = 3  # max distance for centroid polyhedra
coordination_no = 8

center_atom_labels = []
center_atom_positions = []

orbit_atom_labels = []
orbit_atom_positions = []

In [ ]:
for center_label, center_position in zip(orth_labels, orthonormal_positions):
    if center_atom in center_label:
        polyhedra_labels = []
        polyhedra_positions = []
        polyhedra_labels.append(center_label[1])
        polyhedra_positions.append(center_position)
        for orbit_label, orbit_position in zip(orth_labels, orthonormal_positions):
            if orbit_atom in orbit_label and distance_calculator.eucledian_distance(center_position, orbit_position) < max_d:
                polyhedra_labels.append(orbit_label[1])
                polyhedra_positions.append(orbit_position)
        
        
        center_atom_labels.append(polyhedra_labels[0])
        center_atom_positions.append(polyhedra_positions[0])
        
        orbit_atom_labels.append(polyhedra_labels[1:])
        orbit_atom_positions.append(polyhedra_positions[1:])

    


In [ ]:
for i in orbit_atom_positions:
    print(len(i))

NameError: name 'rmc_data' is not defined

In [10]:
atom_list = np.array([['1', 'Sr', '[1]', '0.000000', '0.000000', '0.000000', '1', '0', '0', '0'], 
                      ['2', 'Ti', '[1]', '0.500000', '0.500000', '0.500000', '2', '0', '0', '0'],
                      ['3', 'O', '[1]', '0.500000', '0.500000', '0.000000', '3', '0', '0', '0'],
                      ['4', 'O', '[1]', '0.000000', '0.500000', '0.500000', '4', '0', '0', '0'],
                      ['5', 'O', '[1]', '0.500000', '0.000000', '0.500000', '5', '0','0', '0']])
"""
Orthonormalisation of a set of 3D coordinates.
Coordinates taken from <atom_list>
Original basis taken grom <cell_parameters>
"""
#--------------------------------------------------------------#
# Initialise transformation matrix M
#--------------------------------------------------------------#

supercell_size = [1, 1, 1]

a=4
b=4
c=4
al=90
be=90
ga=90
al = np.deg2rad(al)
be = np.deg2rad(be)
ga = np.deg2rad(ga)

volume = (a * b * c *
          (1 - np.cos(al)**2
           - np.cos(be)**2
           - np.cos(ga)**2
           + 2 * np.cos(al)
           * np.cos(be)
           * np.cos(ga))
          ** 0.5)

a1 = a
a2 = 0
a3 = 0

b1 = b * np.cos(ga)
b2 = b * np.sin(ga)
b3 = 0

c1 = c * np.cos(be)
c2 = c * (np.cos(al) - (np.cos(be) *
                        np.cos(ga))) / np.sin(ga)
c3 = volume / (a * b * np.sin(ga))

M = np.array([[a1, b1, c1], [a2, b2, c2], [a3, b3, c3]])

average_cell_volume = volume / np.prod(supercell_size)

matrix = M

#--------------------------------------------------------------#


#--------------------------------------------------------------#
# Orthonormalisation calculaion
#--------------------------------------------------------------#

atom_positions = atom_list[:, 3:6]

# Make dtype float64
atom_positions = np.float64(atom_positions)

# Orthonormalisation of atomic positions
orthonormal_positions = np.dot(M, atom_positions.T).T

# Round to make very small values (e.g. ~10e-16), zero.
# parse orthonormal positions to class instance
orthonormal_positions = np.around(orthonormal_positions, 8)

In [14]:
np.dot(M, atom_positions.T).T

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.20770554,  0.33643627,  1.04398384],
       [ 1.10385277,  1.78799333,  0.        ],
       [-1.79229446,  0.33643627,  1.04398384],
       [ 1.10385277, -1.45155705,  1.04398384]])